# MNIST example with only tensorflow
copyed by namjoonggu

In [ ]:
%matplotlib inline
%tensorflow_version 1.x

import tensorflow as tf
# tf.reset_default_graph()

TensorFlow 1.x selected.


# Load datasets

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

```mnist = Datasets(train, validation, test)```

인 데이터 구조를 가지고 있다.

Please use alternatives such as official/mnist/dataset.py from tensorflow/models. 이라는 말이 있지만 무시. 

(Datasets라는 class를 변경하라는 뜻이다.)


# Make a model

In [ ]:
# placeholder is used for feeding data.
x = tf.placeholder("float", shape=[None, 784], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target') # shape argument is optional, but this is useful to debug.

tensorflow 2.x에서는 placeholder라는 것이 없어졌다고 한다. [텐서플로 1 코드를 텐서플로 2로 바꾸기](https://www.tensorflow.org/guide/migrate?hl=ko)

기회가 된다면 써보고 싶다.

하지만 그보다는 keras를 사용하는 것이 단순 sequancial 로직에서는 구현이 쉽다.

In [ ]:
# reshape input data
x_image = tf.reshape(x, [-1,28,28,1], name="x_image")

Convolutional Neural Network 모델은 기본적으로 다음과 같은 구조가 한 층이다. 

Convolution (위 과정에서는 bias 존재) -> Activation -> Pooling

In [ ]:
# Build a convolutional layer and maxpooling with random initialization
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1), name="W_conv1") # W is [row, col, channel, feature]
b_conv1 = tf.Variable(tf.zeros([32]), name="b_conv1")
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1, name="h_conv1")
h_pool1 = tf.nn.max_pool( h_conv1 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = "h_pool1")

두 층

In [ ]:
# Repeat again with 64 number of filters
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1), name="W_conv2") # W is [row, col, channel, feature]
b_conv2 = tf.Variable(tf.zeros([64]), name="b_conv2")
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2, name="h_conv2")
h_pool2 = tf.nn.max_pool( h_conv2 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = "h_pool2")

두 번의 Convolution 이후 fully connected layer를 사용하여 우리가 원하는 classification 형태의 답을 도출해낸다.

In [ ]:
# Build a fully connected layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64], name="h_pool2_flat")
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1), name = 'W_fc1')
b_fc1 = tf.Variable(tf.zeros([1024]), name = 'b_fc1')
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name="h_fc1")

# Dropout Layer
keep_prob = tf.placeholder("float", name="keep_prob")
h_fc1 = tf.nn.dropout(h_fc1, keep_prob, name="h_fc1_drop")

# Build a fully connected layer with softmax 
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1), name = 'W_fc2')
b_fc2 = tf.Variable(tf.zeros([10]), name = 'b_fc2')
y=tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2, name="y")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Loss function, Optimizer, Accuracy

Loss function is cross entropy function saved as ```cross_entropy```

Optimizer is Adam saved as ```tf.train.AdamOptimizer```

Accuracy needs ```correct_prediction``` which predict correctness as list of boolean.

In [ ]:
# define the Loss function as "cross entropy function" saved in tensorflow
cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')

# define optimization algorithm as "Gradient descent" or "Adam"
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# correct_prediction is list of boolean which is the result of comparing(model prediction , data)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))

# define accuracy shown for us
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
# tf.cast() : changes true -> 1 / false -> 0
# tf.reduce_mean() : calculate the mean

# Summary

In [ ]:
# create summary of parameters
tf.summary.histogram('weights_1', W_conv1)
tf.summary.histogram('weights_2', W_conv2)
tf.summary.histogram('y', y)
tf.summary.scalar('cross_entropy', cross_entropy)
merged = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("summary")

# Create Session and run

마찬가지로 tensorflow 2.x에서는 Session의 개념이 없어졌다고 한다. [텐서플로 1 코드를 텐서플로 2로 바꾸기](https://www.tensorflow.org/guide/migrate?hl=ko)

Session과 placeholder가 functional program의 특성이라고 하는데 이 또한 공부해보아야 하겠다.

In [ ]:
# Create Session
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True))) as sess:  # open a session which is a envrionment of computation graph.
    sess.run(tf.global_variables_initializer())# initialize the variables


    # training the MLP
    for i in range(5001): # minibatch iteraction
        batch = mnist.train.next_batch(100) # minibatch size
        sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1], keep_prob: 0.5}) # placeholder's none length is replaced by i:i+100 indexes

        if i%500 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1], keep_prob: 1})
            print ("step %d, training accuracy: %.3f"%(i, train_accuracy))

            # calculate the summary and write.
            summary = sess.run(merged, feed_dict={x:batch[0], y_target: batch[1], keep_prob: 1})
            summary_writer.add_summary(summary , i)

    # for given x, y_target data set
    print  ("test accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.test.images[0:250], y_target: mnist.test.labels[0:250], keep_prob: 1}))

step 0, training accuracy: 0.040
step 500, training accuracy: 1.000
step 1000, training accuracy: 0.960
step 1500, training accuracy: 0.990
step 2000, training accuracy: 0.970
step 2500, training accuracy: 0.990
step 3000, training accuracy: 1.000
step 3500, training accuracy: 0.990
step 4000, training accuracy: 1.000
step 4500, training accuracy: 0.990
step 5000, training accuracy: 0.990
test accuracy: 0.996


In [2]:
%cd "/content/drive/My Drive/Colab Notebooks/AI_Vision"
!git add mnist_example_tf_v1.ipynb

/content/drive/My Drive/Colab Notebooks/AI_Vision


In [ ]:
!git commit -m "Retype mnist with CNN"
!git push origin master